# 🧠 Clasificación de Productos - Google Colab
Este notebook carga datos desde Google Drive, entrena una red neuronal convolucional (CNN) y evalúa su desempeño.

In [ ]:
# 📂 Montar Google Drive con dataset ya preprocesado
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 🚀 Clonar repositorio
!git clone https://github.com/solivare/DeepNeuralNetworkUSS.git
%cd DeepNeuralNetworkUSS/Projects/Supermarket/

In [ ]:
# 📦 Instalar dependencias
!pip install -r requirements.txt

In [ ]:
# 📁 Ajustar paths para importar módulos
import os, sys
src_path = os.path.abspath('src')
if src_path not in sys.path:
    sys.path.append(src_path)

## 🧠 Entrenamiento del Modelo

In [ ]:
# 🔧 Cargar config y actualizar ruta a Drive
from src import train, model, utils
import yaml

with open("config.yaml") as f:
    config = yaml.safe_load(f)

# Sobrescribir ruta para correr desde Drive
config["paths"]["prepared_data"] = "/content/drive/MyDrive/DeepNeuralNetworkUSS/Data/Supermarket/items_prepared"

# 🧠 Entrenamiento
train_gen, val_gen = train.get_data_generators(config)
num_classes = len(train_gen.class_indices)

cnn_model = model.build_model(config, num_classes)

history = cnn_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=config['training']['epochs']
)

utils.plot_training_history(history)

# ✍️ Mejora el modelo en src/model.py

## 📊 Evaluación del Modelo

In [ ]:
from src import evaluate as ev
import numpy as np
from sklearn.metrics import classification_report

y_true = val_gen.classes
y_scores = cnn_model.predict(val_gen)
y_pred = np.argmax(y_scores, axis=1)
class_names = list(val_gen.class_indices.keys())

ev.plot_confusion_matrix(y_true, y_pred, class_names)
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# ✍️ TODO: Agrega estas funciones de visualización
# - Distribución de probabilidades
# - F1 vs threshold
# - Imágenes mal clasificadas

# ev.plot_prediction_distribution(y_true, y_scores[:, i], class_name=class_names[i])
# ev.plot_f1_vs_threshold(y_true, y_scores[:, i], modelo="CNN")
# ev.show_misclassified_images(val_gen, cnn_model, class_names)